- Three essential computer vision tasks
- Image segmentation
- Modern convnet architecture patterns
  - Modularity, hierarchy, reuse
  - Residual connections
  - Batch normalizaton
  - Depthwise separable convolution
  - Mini Xception model
- Interpret what convnets learn
  - Visualize intermediate activations
  - Visualize convet filters
  - Visualize heatmaps of class acitvation